In [2]:
import os

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
model = ChatGoogleGenerativeAI(model='gemini-pro', convert_system_message_to_human= True)

In [7]:
response = model.invoke("Hello, world!").content

In [8]:
from langchain_core.output_parsers import StrOutputParser

In [9]:
parser = StrOutputParser()

In [10]:
chain = model | parser 

In [11]:
chain.invoke("Hello, world!")

'Hello, world!\n\nI am a large language model, trained by Google.'

In [12]:
from langchain_core.messages import HumanMessage

In [13]:
while True:
    message = input("Write your query")
    if message == "Bye":
        break
    else:
        print(chain.invoke([HumanMessage(content=message)]))

In [14]:
from langchain_core.chat_history import BaseChatMessageHistory

In [15]:
from langchain_core.chat_history import InMemoryChatMessageHistory

In [16]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [17]:
from langchain_core.messages import AIMessage

In [18]:
output = model.invoke([
        HumanMessage(content="Hi! I'm cristinao"),
        AIMessage(content="Hello cristinao! How can I assist you today?"),
        HumanMessage(content="What's my name?") 
])

In [19]:
parser.invoke(output)

'Your name is cristinao, as you have told me.'

In [20]:
store ={}

In [21]:
def get_session_history(session_id: str) ->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [22]:
model_with_memory = RunnableWithMessageHistory(model, get_session_history)

In [23]:
config = {"configurable": {"session_id": "firstchat"}}

In [24]:
model_with_memory.invoke([HumanMessage(content="Hi! I am Samin Sadaf")], config=config).content

"Hi Samin Sadaf, it's nice to meet you!"

In [25]:
model_with_memory.invoke([HumanMessage(content="What is my name?")], config=config).content

'Your name is Samin Sadaf.'

In [26]:
config = {"configurable": {"session_id": "secondchat"}}

In [27]:
model_with_memory.invoke([HumanMessage(content="What is my name?")], config=config).content

'I do not have access to personal information, including your name.'

In [28]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content="Hi Samin Sadaf, it's nice to meet you!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-31356a5d-23fa-4324-be02-2798a0e8ec2b-0', usage_metadata={'input_tokens': 9, 'output_tokens': 14, 'total_tokens': 23}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'ca

In [29]:
config = {"configurable": {"session_id": "firstchat"}}

In [30]:
model_with_memory.invoke([HumanMessage(content="What did I ask you last time?What was your answer?")], config=config).content

'Last time, you asked me "What is my name?". I answered "Your name is Samin Sadaf."'

In [31]:
config = {"configurable": {"session_id": "secondchat"}}

In [32]:
model_with_memory.invoke([HumanMessage(content="What did I ask you last time?What was your answer?")], config=config).content

'I do not have access to previous conversations or interactions, so I cannot provide information about what you asked me last time or what my answer was.'

In [33]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content="Hi Samin Sadaf, it's nice to meet you!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-31356a5d-23fa-4324-be02-2798a0e8ec2b-0', usage_metadata={'input_tokens': 9, 'output_tokens': 14, 'total_tokens': 23}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'ca

In [34]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [35]:
prompt=ChatPromptTemplate.from_messages(
    [
      ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
      MessagesPlaceholder(variable_name="messages"),
    ]
)

In [36]:
chain = prompt | model | parser

In [37]:
chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

'Hi Bob, how can I help you today?'

In [38]:
chain.invoke({"messages": [HumanMessage(content="What is my name?")]})

'I do not have access to your personal information, including your name.'

In [39]:
chain_with_memory = RunnableWithMessageHistory(chain, get_session_history)

In [40]:
config = {"configurable":{"session_id":"thirdchat"}}

In [41]:
chain_with_memory.invoke({"messages": [HumanMessage(content="Hi! I'm bob")]}, config=config)

"Hello, bob. It's nice to meet you. I am a helpful assistant. I will do my best to answer all of your questions to the best of my ability."

In [42]:
chain_with_memory.invoke({"messages": [HumanMessage(content="What is my name?")]}, config=config)

'I do not have access to your personal information, so I cannot tell you your name. However, you told me that your name is bob.'

In [43]:
chain_with_memory.invoke({"messages": [HumanMessage(content="What is 2+2=?")]}, config=config)

'2 + 2 = 4'

In [44]:
chain_with_memory.invoke(
    [HumanMessage(content="what should i add in previous addition so that i will become 10?"),
     ],config=config
)

'You need to add 4 to the previous addition, 2 + 2, to get 10.\n\n2 + 2 + 4 = 10'

In [45]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content="Hi Samin Sadaf, it's nice to meet you!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-31356a5d-23fa-4324-be02-2798a0e8ec2b-0', usage_metadata={'input_tokens': 9, 'output_tokens': 14, 'total_tokens': 23}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'ca

Trim conversation to be within the context window of the model

In [46]:
from langchain_core.messages import SystemMessage, trim_messages

In [47]:
trimmer = trim_messages(
    max_tokens = 40,
    strategy = 'last',
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = 'human',
)

In [48]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [49]:
model.get_num_tokens_from_messages(messages)

51

In [50]:
trimmed_messages = trimmer.invoke(messages)
trimmed_messages

[HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [51]:
model.get_num_tokens_from_messages(trimmed_messages)

39

In [52]:
trimmed_messages + [HumanMessage(content="What do I like?")]

[HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!'),
 HumanMessage(content='What do I like?')]

In [53]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
    | parser
)

In [54]:
trimmed_messages + [HumanMessage(content="What math problem I ask?")] 

[HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!'),
 HumanMessage(content='What math problem I ask?')]

In [55]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What is the last thing I said?")],
        "language": "English",
    }
)
response

'Having fun?'

In [56]:
model_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [57]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content="Hi Samin Sadaf, it's nice to meet you!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-31356a5d-23fa-4324-be02-2798a0e8ec2b-0', usage_metadata={'input_tokens': 9, 'output_tokens': 14, 'total_tokens': 23}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'ca

In [64]:
config = {"configurable":{"session_id":"fourthchat"}}

In [65]:
model_with_memory.invoke({
    "messages": messages + [HumanMessage(content="What is my name?")],
    "language": "English",
}, config= config)

'I do not have access to that information.'

In [66]:
model_with_memory.invoke({
    "messages": messages + [HumanMessage(content="What is the last thing I asked?")],
    "language": "English",
}, config= config)

'You last asked "having fun?"'

In [67]:
store 

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi! I am Samin Sadaf'), AIMessage(content="Hi Samin Sadaf, it's nice to meet you!", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-31356a5d-23fa-4324-be02-2798a0e8ec2b-0', usage_metadata={'input_tokens': 9, 'output_tokens': 14, 'total_tokens': 23}), HumanMessage(content='What is my name?'), AIMessage(content='Your name is Samin Sadaf.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'ca